In [1]:
import gensim
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
import nltk
import os
import re
import json

import math

C:\Users\piyus_000\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
data_folder = "data/NewsArticles/"
files = os.listdir(data_folder)

# Preprocessing

In [3]:
s = set(nltk.corpus.stopwords.words('english'))
s.add('')

In [4]:
texts = []
for file in files:
    with open(data_folder+file,encoding="ISO-8859-1") as f:
        text = f.read()
        text = re.split('[, \.\n]', text)
        text = [word.lower() for word in text if word.lower() not in s and word.isalpha() and len(word)>=3]
        texts.append(text)

In [5]:
common_dictionary = Dictionary(texts)

In [6]:
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

# Training

In [7]:
lda = gensim.models.LdaModel(common_corpus,id2word=common_dictionary ,num_topics=30)

In [8]:
lda.get_document_topics(common_dictionary.doc2bow(texts[1]))

[(11, 0.990136)]

# Text v/s Topic

In [9]:
matrix = [[str(0) for i in range(150)] for i in range(len(texts))]

### Increase for more Radius
factor = 1
###

for i in range(len(texts)):
    topics = lda.get_document_topics(common_dictionary.doc2bow(texts[i]))
    for t in topics:
        matrix[i][t[0]] = str(t[1] * factor)

In [10]:
string = ','+','.join(["Topic"+str(i) for i in range(30)])
for i in range(len(texts)):
    string += '\ndoc' + str(i) + ',' + ','.join(matrix[i])

with open("data/text_topic_matrix.csv", "w") as f:
    f.write(string)

# Word v/s Topic

In [11]:
csv_string = 'word,value'

for t in lda.get_topic_terms(1, topn=100):
#     print( lda.id2word[t[0]],t[1],t[0])
    csv_string += '\n' + lda.id2word[t[0]] + ',' + str(t[1])
# t = lda.get_topic_terms(57, topn=100)

with open('topic1.csv', 'w') as f:
    f.write(csv_string)

In [12]:
words = {}
for k,v in lda.id2word.items():
    words[v] = k
len(words)

1715

In [13]:
word_dict = {}
for topic in range(30):
    word_list = lda.get_topic_terms(topic, topn=100)
    for word_tuple in word_list:
        word_id = word_tuple[0]
        word_score = word_tuple[1]
        if word_id not in word_dict:
            word_dict[word_id] = {}
        word_dict[word_id][topic] = word_score

# DOC TEXT

In [14]:
texts = []
ix = 0
for file in files:
    print(ix)
    with open(data_folder+file,encoding="ISO-8859-1") as f:
        text = "\n"+f.read()
        text_words = set(re.split('[, \.\n]', text))
        html_string = text
        for word in text_words:
            if word.lower() in words and words[word.lower()] in word_dict:
                keys = list(word_dict[words[word.lower()]].keys())
                classes = ''
                for i in keys:
                    classes += 't'+str(i)+" "
#                 html_string = html_string.replace(r"\b"+word+"\b","<span class=\""+classes+"hoverable\">"+word+"</span>")
                html_string = html_string.replace("\n"+word+" ","\n<span class=\""+classes+"hoverable\">"+word+"</span> ")
                html_string = html_string.replace(" "+word+"\n"," <span class=\""+classes+"hoverable\">"+word+"</span>\n")
                html_string = html_string.replace(" "+word+" "," <span class=\""+classes+"hoverable\">"+word+"</span> ")
                html_string = html_string.replace("\n"+word+"\n","\n<span class=\""+classes+"hoverable\">"+word+"</span>\n")
#                 html_string = re.sub(word+" ","<span class=\""+classes+"hoverable\">"+word+"</span>",html_string)
#                 html_string = re.sub(" "+word,"<span class=\""+classes+"hoverable\">"+word+"</span>",html_string)
        with open("data/doc"+str(ix)+".txt","w",encoding="ISO-8859-1") as f1:
            f1.write(html_string)
    ix+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


# TOPIC V/S WORDS

In [15]:
for topic in range(30):
    word_list = lda.get_topic_terms(topic, topn=100)
    file_str = "word,value"
    for word_tuple in word_list:
        word_id = word_tuple[0]
        file_str += "\n" + str(lda.id2word[word_tuple[0]]) +","+ str(word_tuple[1])
    with open("data/topic"+str(topic)+".csv", "w") as f:
        f.write(file_str)

In [16]:
files = sorted(files)
doc_id = {}
for i in range(len(files)):
    doc_id['doc'+str(i)] = files[i]
with open('data/doc_ids.json','w') as f:
    json.dump(doc_id, f, indent=True)

NameError: name 'json' is not defined